# الجلسة 1 – بدء تشغيل الدردشة (Foundry Local)

يقوم هذا الدفتر ببدء تشغيل Foundry Local، وتنزيل الاسم المستعار للنموذج المفضل، وتنفيذ إكمال الدردشة القياسي والمتدفق.


# السيناريو
تقدم هذه الجلسة الحد الأدنى المطلوب لجعل نموذج لغة صغير محلي يستجيب عبر Foundry Local. ستقوم بـ:
- تثبيت SDK / تبعيات العميل.
- تهيئة مدير Foundry Local لاسم مستعار مختار (الافتراضي: `phi-3.5-mini`).
- تطبيق تصحيح دفاعي لتحمل الحقول الاختيارية في بيانات تعريف النموذج.
- إرسال طلب إكمال محادثة قياسي.
- بث الاستجابة رمزًا تلو الآخر.

الهدف هو التحقق من تشغيلك المحلي ومسار الشبكة قبل الانتقال إلى RAG أو التوجيه أو الوكلاء.


### شرح: تثبيت التبعيات
تثبيت حزم Python المطلوبة لتشغيل هذا التدفق البسيط للدردشة:
- `foundry-local-sdk`: إدارة النماذج المحلية ودورة حياة الخدمات.
- `openai`: واجهة مألوفة لإكمال الدردشة.
- `rich`: طباعة جميلة لتحسين وضوح المخرجات في الدفاتر.

إعادة التشغيل آمنة (غير متكررة). يمكن تخطي هذه الخطوة إذا كانت بيئتك تحتوي بالفعل على هذه الحزم.


In [1]:
# Install required libraries (idempotent)
!pip install -q foundry-local-sdk openai rich

### شرح: الواردات الأساسية
يجلب الوحدات المستخدمة في جميع أنحاء الدفتر:
- `FoundryLocalManager` للتفاعل مع بيئة تشغيل النموذج المحلي.
- عميل `OpenAI` حتى نتمكن من إعادة استخدام واجهة API المألوفة لإكمال الدردشة.
- `rich.print` للإخراج المزخرف.

لا تحدث أي مكالمات شبكية هنا—هذا فقط لتحضير مساحة الأسماء.


In [2]:
import os
from foundry_local import FoundryLocalManager
from openai import OpenAI
from rich import print

### شرح: تهيئة المدير وتعديل البيانات الوصفية
يقوم بتهيئة `FoundryLocalManager` للاسم المستعار المختار ويطبق تعديلًا دفاعيًا لمعالجة استجابات الخدمة بشكل سلس حيث قد يكون `promptTemplate` قيمته `null`.

النتائج الرئيسية:
- تأكيد حالة الخدمة ونقطة النهاية.
- عرض النماذج المخزنة مؤقتًا (التحقق من التخزين المحلي).
- تحديد معرف النموذج المحدد للاسم المستعار (يُستخدم في مكالمات الدردشة لاحقًا).

إذا واجهت مشاكل في التحقق من صحة البيانات الوصفية الخام للخدمة، يوضح هذا النمط كيفية التنقية دون الحاجة إلى تعديل SDK.


In [3]:
# Catalog-safe manager initialization (handles null promptTemplate values)
import os
from foundry_local import FoundryLocalManager
from foundry_local.models import FoundryModelInfo
from openai import OpenAI
from rich import print

# Monkeypatch to tolerate service responses where promptTemplate is null
_original_from_list_response = FoundryModelInfo.from_list_response

def _safe_from_list_response(response):  # type: ignore
    try:
        if isinstance(response, dict) and response.get("promptTemplate") is None:
            # Normalize to empty dict so pydantic validation passes
            response["promptTemplate"] = {}
    except Exception as e:  # pragma: no cover
        print(f"[yellow]Warning: safe wrapper encountered issue normalizing promptTemplate: {e}[/yellow]")
    return _original_from_list_response(response)

# Apply patch only once
if getattr(FoundryModelInfo.from_list_response, "__name__", "") != "_safe_from_list_response":
    FoundryModelInfo.from_list_response = staticmethod(_safe_from_list_response)  # type: ignore

ALIAS = os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-3.5-mini')
manager = FoundryLocalManager(ALIAS)
print(f'[bold green]Service running:[/bold green] {manager.is_service_running()}')
print(f'Endpoint: {manager.endpoint}')
print('Cached models:', manager.list_cached_models())
model_id = manager.get_model_info(ALIAS).id
print(f'Using model id: {model_id}')

Service running: True

Endpoint: http://127.0.0.1:57127/v1

Cached models:
[
    FoundryModelInfo(
        alias='gpt-oss-20b',
        id='gpt-oss-20b-cuda-gpu:1',
        version='1',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/gpt-oss-20b-cuda-gpu/versions/1',
        file_size_mb=9882,
        prompt_template={},
        provider='AzureFoundry',
        publisher='Microsoft',
        license='apache-2.0',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='phi-3.5-mini',
        id='Phi-3.5-mini-instruct-cuda-gpu:1',
        version='1',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/Phi-3.5-mini-instruct-cuda-gpu/versions/1',
        file_size_mb=2181,
        prompt_template={
            'prompt': '<|user|>\n{Content}<|end|>\n<|assistant|>',
            'assistant': '<|assistant|>\n{Content}<|end|>'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='MIT',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='phi-4-mini',
        id='Phi-4-mini-instruct-cuda-gpu:4',
        version='4',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/Phi-4-mini-instruct-cuda-gpu/versions/4',
        file_size_mb=3686,
        prompt_template={
            'system': '<|system|>{Content}<|end|>',
            'user': '<|user|>{Content}<|end|>',
            'assistant': '<|assistant|>{Content}<|end|>',
            'prompt': '<|user|>{Content}<|end|><|assistant|>'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='MIT',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='qwen2.5-0.5b',
        id='qwen2.5-0.5b-instruct-cuda-gpu:3',
        version='3',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/qwen2.5-0.5b-instruct-cuda-gpu/versions/3',
        file_size_mb=528,
        prompt_template={
            'system': '<|im_start|>system\n{Content}<|im_end|>',
            'user': '<|im_start|>user\n{Content}<|im_end|>',
            'assistant': '<|im_start|>assistant\n{Content}<|im_end|>',
            'prompt': '<|im_start|>user\n{Content}<|im_end|>\n<|im_start|>assistant'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='apache-2.0',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='qwen2.5-7b',
        id='qwen2.5-7b-instruct-cuda-gpu:3',
        version='3',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/qwen2.5-7b-instruct-cuda-gpu/versions/3',
        file_size_mb=4843,
        prompt_template={
            'system': '<|im_start|>system\n{Content}<|im_end|>',
            'user': '<|im_start|>user\n{Content}<|im_end|>',
            'assistant': '<|im_start|>assistant\n{Content}<|im_end|>',
            'prompt': '<|im_start|>user\n{Content}<|im_end|>\n<|im_start|>assistant'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='apache-2.0',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='qwen2.5-coder-7b',
        id='qwen2.5-coder-7b-instruct-cuda-gpu:3',
        version='3',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/qwen2.5-coder-7b-instruct-cuda-gpu/versions/3',
        file_size_mb=4843,
        prompt_template={
            'system': '<|im_start|>system\n{Content}<|im_end|>',
            'user': '<|im_star

Using model id: Phi-4-mini-instruct-cuda-gpu:4

### شرح: إكمال الدردشة الأساسي
يقوم بإنشاء عميل متوافق مع `OpenAI` يشير إلى نقطة النهاية المحلية لـ Foundry ويُجري عملية إكمال دردشة واحدة غير متدفقة. التركيز هنا:
- التأكد من أن النموذج يستجيب دون أخطاء.
- التحقق من زمن الاستجابة / تنسيق الإخراج.
- الحفاظ على قيمة `max_tokens` منخفضة لتوفير الموارد.

إذا فشلت هذه العملية، تحقق مرة أخرى من أن خدمة Foundry Local تعمل وأن الاسم المستعار يتم حله بشكل صحيح.


In [4]:
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')
prompt = 'List two benefits of local inference for privacy.'
resp = client.chat.completions.create(model=model_id, messages=[{'role':'user','content':prompt}], max_tokens=120, temperature=0.5)
print(resp.choices[0].message.content)

1. Enhanced Data Privacy: Local inference allows data to be processed locally on a user's device rather than being 
sent to a central server. This reduces the risk of sensitive information being intercepted or exposed during 
transmission, thereby enhancing privacy.

2. Reduced Data Collection: By performing inference locally, the need for extensive data collection is minimized. 
This means that less personal information needs to be stored or transmitted, further protecting user privacy.

### شرح: إكمال الدردشة عبر البث
يوضح كيفية بث الرموز لتحسين زمن الاستجابة الظاهري وتجربة المستخدم التفاعلية. يقوم الحلقة بطباعة التغييرات التدريجية فور وصولها:
- مفيد لواجهات الدردشة حيث يكون للإخراج الجزئي المبكر أهمية.
- يتيح لك قياس معدل بث الرموز مقابل زمن استكمال الإخراج بالكامل.

يمكنك تعديل هذا النمط لتجميع الرموز، تحديث عنصر واجهة التقدم، أو إيقاف العملية أثناء التوليد.


In [5]:
# Streaming example
stream = client.chat.completions.create(model=model_id, messages=[{'role':'user','content':'Give a one-sentence definition of edge AI.'}], stream=True, max_tokens=60, temperature=0.4)
for chunk in stream:
    delta = chunk.choices[0].delta
    if delta and delta.content:
        print(delta.content, end='', flush=True)
print()

Edge

AI

refers

to

artificial

...

with

a

central

server

.


---

**إخلاء المسؤولية**:  
تم ترجمة هذا المستند باستخدام خدمة الترجمة بالذكاء الاصطناعي [Co-op Translator](https://github.com/Azure/co-op-translator). بينما نسعى لتحقيق الدقة، يرجى العلم أن الترجمات الآلية قد تحتوي على أخطاء أو معلومات غير دقيقة. يجب اعتبار المستند الأصلي بلغته الأصلية المصدر الرسمي. للحصول على معلومات حاسمة، يُوصى بالاستعانة بترجمة بشرية احترافية. نحن غير مسؤولين عن أي سوء فهم أو تفسيرات خاطئة ناتجة عن استخدام هذه الترجمة.
